In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import tensorflow as tf

from sklearn.metrics import mean_squared_error

from aepy.data import utils
from aepy.data.datasets import load_CMAPSS
from aepy.data.preprocessing import CMAPSS_preprocessor
from aepy.data.utils import evaluate
from aepy.metrics import cmapps_score
from aepy.models.base import RecurrentDecoder, RecurrentEncoder
from aepy.models.vae import VAE
from aepy.pipelines import PreprocessPipeline, TrainingPipeline

2023-11-21 10:25:05.528494: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow backend


In [2]:
# Check GPU and Tensorflow version
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

print(tf.__version__)

No GPU found
2.16.0-dev20231103


In [3]:
dataset = 'FD003'
# sensors to work with: T30, T50, P30, PS30, phi
sensors = ['s_3', 's_4', 's_7', 's_11', 's_12']
# windows length
sequence_length = 30
# smoothing intensity
alpha = 0.1
# max RUL
threshold = 125

# NOT IMPLEMENTED
# x_train, y_train, x_val, y_val, x_test, y_test = utils.get_data(dataset, sensors,
# sequence_length, alpha, threshold)
train, test, y_test = load_CMAPSS('FD001')
# x_train, y_train, x_val, y_val, x_test, y_test = CMAPSS_preprocessor(train, test, y_test)
preprocess_pipeline = PreprocessPipeline(
    name='CMAPPS_preprocessing', preprocessor=CMAPSS_preprocessor)
x_train, y_train, x_val, y_val, x_test, y_test = preprocess_pipeline(
    train=train, test=test, y_test=y_test, threshold=100)

2023-11-21 10:25:17 [INFO]: +++ CMAPPS_preprocessing +++
2023-11-21 10:25:17 [INFO]: Creating folder in ../output_dir/CMAPPS_preprocessing_2023-11-21_10-25-17
2023-11-21 10:25:17 [INFO]: Selected preprocessor is a function.


In [4]:
timesteps = x_train.shape[1]
input_dim = x_train.shape[2]
intermediate_dim = 300
batch_size = 128
latent_dim = 2
epochs = 2
optimizer = 'adam'

In [5]:
train_data = dict(data=x_train, labels=y_train)
eval_data = dict(data=x_val, labels=y_val)

In [6]:
model = VAE(input_dim=(x_train.shape[1], x_train.shape[2]), latent_dim=2, 
            downstream_task='regression', encoder=RecurrentEncoder, decoder=RecurrentDecoder)
# model_callbacks = utils.get_callbacks("p", model, x_train, y_train)

2023-11-21 10:25:40 [WARNING]: No specific layer configuration has been provided. Creating default configuration...
2023-11-21 10:25:40 [INFO]: Regressor available for the latent space of the autoencoder


In [7]:
pipeline = TrainingPipeline(model=model, num_epochs=1)
trained_model = pipeline(train_data, eval_data)  # , callbacks=model_callbacks)

2023-11-21 10:25:55 [INFO]: +++ TrainingPipeline +++
2023-11-21 10:25:55 [INFO]: Creating folder in ../output_dir/TrainingPipeline_2023-11-21_10-25-55


Cause: Unable to locate the source code of <function _gcd_import at 0x7f2970363400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function _gcd_import at 0x7f2970363400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @t

2023-11-21 10:27:10.497645: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f28640164b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-21 10:27:10.497700: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-21 10:27:11.024224: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1700558839.633657   11110 device_compiler.h:187] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-11-21 10:27:19.670815: E external/local_xla/xla/stream_executor/stream_executor_internal.h:181] SetPriority unimplemented for this stream.



Epoch 1: val_total_loss improved from inf to 0.00000, saving model to ../output_dir/TrainingPipeline_2023-11-21_10-25-55/model.weights.h5
112/112 - 242s - 2s/step - kl_loss: 108.7285 - reconstruction_loss: 0.3492 - reg_loss: 4352.5679 - total_loss: 4461.6450 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_reg_loss: 0.0000e+00 - val_total_loss: 0.0000e+00


In [8]:
test_data = dict(data=x_test, labels=y_test)
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.expand_dims(test_data['labels'], axis=-1),
         y_hat=y_hat['reg'], sel_metric=mean_squared_error)
evaluate(y_true=np.expand_dims(test_data['labels'], axis=-1),
         y_hat=y_hat['reg'], sel_metric=cmapps_score.CMAPSS_Score())

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-11-21 10:52:57 [WARNING]: No specific layer configuration has been provided. Creating default configuration...


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-11-21 10:53:00 [WARNING]: No encoder provided, using default MLP encoder


ValueError: Exception encountered when calling VAE.call().

[1min user code:

    File "/home/lucas/experiment/aepy/examples/../aepy/models/vae/vae_model.py", line 32, in __init__  *
        BaseAE.__init__(self, input_dim, latent_dim,
    File "/home/lucas/experiment/aepy/examples/../aepy/models/base/base_model.py", line 54, in __init__  *
        self.encoder = Encoder_MLP(self.input_dim, self.latent_dim, self.layers_conf)
    File "/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/keras_core/src/layers/layer.py", line 1250, in __setattr__
        value = self._tracker.track(value)
    File "/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/keras_core/src/utils/tracking.py", line 74, in track
        self.add_to_store(name, attr)
    File "/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/keras_core/src/utils/tracking.py", line 97, in add_to_store
        raise ValueError(self._lock_violation_msg)

    ValueError: You cannot add new elements of